In [26]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import urllib.request
#!python -m textblob.download_corpora #this command is only work in jupyter notebook/Ipython
from textblob import TextBlob
from bs4 import BeautifulSoup

In [5]:
data = pd.read_csv('C:\\Users\\Arpit14276\\Desktop\\Rel infra\\userfeedback.csv')
# Keeping only the neccessary columns
data = data[['Review','Sentiment']]

In [7]:
data = data[data.Sentiment != "Neutral"]
data['Review'] = data['Review'].apply(lambda x: x.lower())
data['Review'] = data['Review'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

D:\Product\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\Product\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
print(data[ data['Sentiment'] == 'Positive'].size)
print(data[ data['Sentiment'] == 'Negative'].size)

74470
23266


In [10]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['Review'].values)
X = tokenizer.texts_to_sequences(data['Sentiment'].values)
X = pad_sequences(X)

In [11]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1, 128)            256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 1, 128)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
Y = pd.get_dummies(data['Sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(32741, 1) (32741, 2)
(16127, 1) (16127, 2)


In [13]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
 - 10s - loss: 0.0167 - acc: 0.9968
Epoch 2/7
 - 9s - loss: 2.0964e-05 - acc: 1.0000
Epoch 3/7
 - 9s - loss: 6.6197e-06 - acc: 1.0000
Epoch 4/7
 - 9s - loss: 2.7880e-06 - acc: 1.0000
Epoch 5/7
 - 10s - loss: 1.3782e-06 - acc: 1.0000
Epoch 6/7
 - 9s - loss: 7.6271e-07 - acc: 1.0000
Epoch 7/7
 - 9s - loss: 4.6415e-07 - acc: 1.0000


In [14]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.00
acc: 1.00


In [15]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 100.0 %
neg_acc 100.0 %


In [28]:
def web_crawl(keyword_search):
    url_base='https://news.google.com/search?q='+'&'.join(keyword_search.split())+'&hl=en-IN&gl=IN&ceid=IN%3Aen'
    print ('url', url_base)
    page = urllib.request.urlopen(url_base)
    soup = BeautifulSoup(page, 'html.parser')
    top_10=[]
    for i in  soup.find_all('p')[:10]:
        top_10.append(i.text)
    return top_10
def sentiment (text):
    for i in text:
        #twt = ["After reaching as high as Rs.1126.75 and dropping as low as Rs.1111.75 through the day, one stock of Reliance Industries Ltd. was observed to be priced at "]
        #vectorizing the tweet by the pre-fitted tokenizer instance
        twt = tokenizer.texts_to_sequences(i)
        #padding the tweet to have exactly the same shape as `embedding_2` input
        twt = pad_sequences(twt, maxlen=1, dtype='int32', value=0)
        #print(twt)
        sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
        print ('NEWS : ',i)
        if(np.argmax(sentiment) == 0):
            print("negative")
        elif (np.argmax(sentiment) == 1):
            print("positive")

In [29]:
sentiment(web_crawl('Mastek'))

url https://news.google.com/search?q=Mastek&hl=en-IN&gl=IN&ceid=IN%3Aen
NEWS :  Its consolidated income from operations was up by 31.5 percent to Rs 244 crore in April-June 2018 quarter as against Rs 185.6 crore in the year-ago period.
negative
NEWS :  Read more about Mastek standalone net profit declines 0.11% in the June 2018 quarter on Business Standard. Sales rise 21.67% to Rs 47.38 crore.
positive
NEWS :  Mastek Ltd's Wholly Owned Subsidiary - Mastek (UK) Limited has entered into an arrangement with its Wholly Owned Subsidiary - IndigoBlue Consulting Limited ...
positive
NEWS :  The *content* of this article will benefit those of you who are starting to educate yourself about investing in the stock market and want a simplistic look at the return ...
positive
NEWS :  Follow the latest in business and stock markets on BloombergQuint. Our digital streaming *service* includes comprehensive live programming on a daily basis, ...
positive
NEWS :  Mastek Limited (NSE:MASTEK), is a ₹11.90